In [ ]:
import jsonify
import flask
from flask import Flask
import os
import glob
import pymongo
from pymongo import MongoClient
import bson.binary
import bson.objectid
import bson.errors
from PIL import Image
import joblib 
import numpy as np
import pymysql
import pymysql.cursors
from  bson.json_util import dumps
#與 Mysql連線
connection = pymysql.connect(host='mysql',
                             user='root',
                             password='iii',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

#與 MongoDB連線
client = pymongo.MongoClient(host="mongodb", port=27017)
db = client.image
myset = db.img




app = Flask(__name__)
@app.route("/",methods=["POST"])
def recommend():
    if flask.request.method == "POST":
        userdata=flask.request.json
    print(userdata)
    cluster=clf.predict(np.array([userdata['age'],userdata['been2tko'],
                                  userdata['days'],userdata['budget'],userdata['isFemale']]).reshape((1,-1)))[0]
                        #omiyage variables: 年齡, 去過次數, 去幾天 , 總預算, 性別-男:0；女:1
    print(cluster)
    queryResult=[]
    distribution_0={'餅乾類':3,'小蛋糕類':2,'點心類':3,'巧克力類':1,'夾心餅乾類':1}
    distribution_1={'餅乾類':3,'小蛋糕類':2,'點心類':1,'巧克力類':2,'夾心餅乾類':2}
    distribution_2={'餅乾類':3,'小蛋糕類':1,'點心類':2,'巧克力類':2,'夾心餅乾類':2}

    if cluster == 0:
        distribution=distribution_0
    elif cluster == 1:
        distribution=distribution_1
    else:
        distribution=distribution_2

    #列出目錄下的所有圖片
    files = os.listdir("/home/jovyan/work/Souvenir_pic")
    # print(files[1:])
    for i,p in enumerate(files[1:]):
        files_1 = glob.glob("/home/jovyan/work/Souvenir_pic/" + p + "/*.jpg")
    #     print(files_1)
        #遍歷圖片目錄集合
        for file in files_1:
    #         print(file)
            #圖片的全路徑
            filename=file.split('/')[-1]
    #         print(filename)

        f = open(file, 'rb')
    #     print(f)
    #     print(type(f))

        #轉換為mongodb二進位數據儲存形式
        content = bson.binary.Binary(f.read())
    #     print(type(content))
        f.close()
        #插入到數據庫
        myset.insert({'filename':filename,'data':content})
    #     print(myset)
    #     print(type(myset))

        #提取資料
        data = myset.find_one({'filename':filename})
    #     print(type(data['filename']))
    #     print(data['filename']) 


    with connection.cursor() as cursor:
        sql = "USE`test`"
        cursor.execute(sql)
        for kind in list(distribution.keys()): 
            cursor.execute( "SELECT * FROM `trip4.csv` WHERE Types= '"+kind+"' ORDER BY RAND() LIMIT "+str(distribution[kind]))          
            queryResult+=cursor.fetchall()
#     print(len(queryResult))
    print(queryResult)
    Data = []
    for i in queryResult:
        
        imgname = i['souvenir'] + ".jpg"
        print(imgname)
        #提圖片檔資料        
        data = myset.find_one({'filename':imgname})
#     print(data)
        Data.append(data)
    print(len(Data)) #幾張推薦商品圖片
    # print(Data)
#         data = myset.find_one({'filename':imgname})
#         print(data['filename'])
#         data = myset.find({'filename':filename})
#         print(data)
#         for s in data:
#             print(s)
    return dumps({'justakey':Data})

if __name__ == "__main__":
    clf = joblib.load('omiyage_clf.joblib')
    
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


{'age': 30, 'been2tko': 4, 'days': 16, 'budget': 50000, 'isFemale': 1}
1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[{'Types': '餅乾類', 'price': 1250, 'ranking': 41, 'souvenir': '東京カンパネラ「東京カンパネラ ショコラ」', 'address': '東京駅\u3000大丸東京 4F', 'url': 'http://hacci1912.com/'}, {'Types': '餅乾類', 'price': 1080, 'ranking': 24, 'souvenir': '大江戸菓子匠\u3000銀座松崎煎餅\u3000「瓦煎餅三味胴」', 'address': '銀座駅周辺\u3000東京都中央区銀座5-6-9松崎ビル1階', 'url': 'http://terraconfect.tokyo/fromageterra/'}, {'Types': '餅乾類', 'price': 450, 'ranking': 60, 'souvenir': 'みりん堂「スカイツリーせんべい」', 'address': '東京スカイツリー周辺\u3000東京都墨田区琴平1-13-7', 'url': 'http://mirindo.com/'}, {'Types': '小蛋糕類', 'price': 411, 'ranking': 69, 'souvenir': 'クラブハリエ B-スタジオ「バームクーヘンmini」', 'address': '池袋駅\u3000東武百貨店池袋店 B1F', 'url': 'http://clubharie.jp/baumkuchen/'}, {'Types': '小蛋糕類', 'price': 486, 'ranking': 21, 'souvenir': 'TAICHIRO MORINAGA「キャラメルラスク」', 'address': '新宿駅\u3000NEWoman（ニュウマン）エキナカ 2F', 'url': 'http://www.morinaga.co.jp/taichiro/'}, {'Types': '點心類', 'price': 1080, 'ranking': 59, 'souvenir': 'ブールミッシュ「パンダのマカロン」', 'address': '上野駅\u3000エキュート上野 3F', 'url': 'http://www.boulmich.co.jp'}, {'Typ

...

10
{'age': 30, 'been2tko': 4, 'days': 16, 'budget': 50000, 'isFemale': 1}
1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:76: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[{'Types': '餅乾類', 'price': 580, 'ranking': 61, 'souvenir': 'ドルチェフェリーチェ「うえきゅんのキャンダック」', 'address': '上野駅\u3000エキュート上野 3F', 'url': 'https://www.keyuca.com/dolce-felice/'}, {'Types': '餅乾類', 'price': 1080, 'ranking': 67, 'souvenir': '銀座 菊廼舎（きくのや）「冨貴寄-ふきよせ-」', 'address': '渋谷駅\u3000渋谷マークシティB1F', 'url': 'http://www.testarossacafe.net/'}, {'Types': '餅乾類', 'price': 617, 'ranking': 23, 'souvenir': 'スイーツハートツリー「いちごのシャリーメイト」', 'address': '東京スカイツリー\u3000東京ソラマチ 2F', 'url': 'http://terraconfect.tokyo/fromageterra/'}, {'Types': '小蛋糕類', 'price': 1944, 'ranking': 31, 'souvenir': '治一郎「バウムクーヘン」', 'address': '羽田空港\u3000第1旅客ターミナル 2F PIER 4', 'url': 'http://www.jiichiro.com'}, {'Types': '小蛋糕類', 'price': 2484, 'ranking': 58, 'souvenir': '日本茶菓SANOAH「キャラメルサンドクッキー 胡桃抹茶」', 'address': '新宿駅\u3000伊勢丹 B1F', 'url': 'http://www.n-sanoah.jp/product/itm-sta-ks'}, {'Types': '點心類', 'price': 573, 'ranking': 90, 'souvenir': '菓匠 青柳正家「菊最中」', 'address': '浅草駅\u3000エキミセ 1F', 'url': 'https://www.aoyagiseike.jp/shop/'}, {'Types': '巧克

127.0.0.1 - - [09/Nov/2019 07:07:05] "POST / HTTP/1.1" 200 -


10


In [ ]:
#MongoDB關閉連線
client.close()

#Mysql關閉連線
connection.close()
